# Feature Selection and Analysis

In this notebook, we will perform various forms of analysis to determine which features that are available to us may be most useful when training multivariate models.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from Functions import tsPlot
from copy import deepcopy as dc
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from skopt import gp_minimize
from tqdm import tqdm  # for progress bar
from IPython.display import clear_output, display

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.inspection import permutation_importance

### Enable GPU

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

### Import Data

In [3]:
# Read the csv file into a pandas DataFrame
df_ret = pd.read_csv('../DataManagement/daily_data.csv', parse_dates=['DATE'], index_col='DATE')

# Specify the date range
start_date = '2018-06-30'
end_date = '2023-06-30'

# Slice the DataFrame for the desired date range
df_ret = df_ret.loc[start_date:end_date].copy()

## Data Pre-Processing

To prepare the data, we follow the exact same steps as we did when building our multivariate LSTM model.

### Create Sequences

Here we define a function that transforms the data into the format requred by the LSTM with parameters for lookback period.

In [4]:
def prepare_data_lstm(data, n_steps, column):
    column_names = [column]
    data = dc(data)  # make deep copy of the input data

    for i in range(1, n_steps+1):
        column_name = f'{column}(t-{i})'
        column_names.append(column_name)
        data[column_name] = data[column].shift(i)

    data.dropna(inplace=True)
    data = data.loc[:, data.columns.intersection(column_names)]

    return data

Now, since we are working with multivariate data, we need to be careful to make sure that our X matrix follows the structure laid out in the diagram found in the README file. We will effectively call the prepare_data_lstm function on each feature seperaterly to generate the sequences and then perform some transformations afterwards to combine the data into a single X matrix as needed.

In [5]:
lookback = 7

# Same as the univariate case
shifted_close = prepare_data_lstm(df_ret, lookback, 'NVDA_CLOSE')    

# New: now we also perform the same procedure on each of the additional features that we wish to include in our X matrix
shifted_open = prepare_data_lstm(df_ret, lookback, 'NVDA_OPEN')
shifted_high = prepare_data_lstm(df_ret, lookback, 'NVDA_HIGH')
shifted_low = prepare_data_lstm(df_ret, lookback, 'NVDA_LOW')
shifted_volume = prepare_data_lstm(df_ret, lookback, 'NVDA_VOLUME')

# Now we convert the dataframes into numpy matrices
shifted_close_np = shifted_close.to_numpy()
shifted_open_np = shifted_open.to_numpy()
shifted_high_np = shifted_high.to_numpy()
shifted_low_np = shifted_low.to_numpy()
shifted_volume_np = shifted_volume.to_numpy()

Now we perform standardisation on the dataset. We have to be careful here because some variables are recorded on very different scales, for example, volume figures are magnitudes larger than open, high, low and close figures. As a result, we will scale volume seperately to the other variables so that we do not introduce bias into the dataset.

In [6]:
# Scaler for price data
price_scaler = MinMaxScaler(feature_range=(-1,1))    # scale to range -1 to 1

# Scaler for volume data
volume_scaler = MinMaxScaler(feature_range=(-1,1))    # scale to range -1 to 1

# Scale the price data
shifted_open_np_scaled = price_scaler.fit_transform(shifted_open_np)
shifted_high_np_scaled = price_scaler.fit_transform(shifted_high_np)
shifted_low_np_scaled = price_scaler.fit_transform(shifted_low_np)
shifted_close_np_scaled = price_scaler.fit_transform(shifted_close_np)

# Scale the volume data
shifted_volume_np_scaled = volume_scaler.fit_transform(shifted_volume_np)

Here we do a quick check to make sure the shapes of out numpy matrices match

In [7]:
print(shifted_close_np_scaled.shape, shifted_volume_np_scaled.shape)

(1251, 8) (1251, 8)


Now we split the standardised returns into our X and y vectors

In [8]:
# Our y vector does not change in the multivariate case since we are still predicting the close prices
y = shifted_close_np_scaled[:, 0]

# Our X matrix does change though as we need to add additional dimensions to store the extra variables
# We start by slicing out the time t column from each of the X components
X_close = shifted_close_np_scaled[:, 1:]
X_open = shifted_open_np_scaled[:, 1:]
X_high = shifted_high_np_scaled[:, 1:]
X_low = shifted_low_np_scaled[:, 1:]
X_volume = shifted_volume_np_scaled[:, 1:]

# Then we individually "flip" each X component so that it goes, for example, t-7, t-6, t-5....
X_close = dc(np.flip(X_close, axis=1))
X_open = dc(np.flip(X_open, axis=1))
X_high = dc(np.flip(X_high, axis=1))
X_low = dc(np.flip(X_low, axis=1))
X_volume = dc(np.flip(X_volume, axis=1))

# Finally, we combine each X component into a single X matrix with the shape (samples, time steps, features) i.e. (1251, 7, 4) in this case
X = np.stack((X_close, X_open, X_high, X_low, X_volume), axis=-1)

# Also, y currently has shape (1251) but it needs to have shape (1251, 1) in this framework
y = y.reshape((-1, 1))

Here we check that the shape of y and X are as expected

In [9]:
y.shape, X.shape

((1251, 1), (1251, 7, 5))

Now perform a train/test split where the first 95% of the data is used for training/validation and the last 5% for testing

In [10]:
# Define the split proportions first
train_ratio = 0.80  # 80% of data for training
valid_ratio = 0.15  # 15% of data for validation
test_ratio = 0.05   # 5% of data for testing

# First split: separate out the test set
train_valid_index = int(len(X) * (train_ratio + valid_ratio))
X_train_valid, X_test = X[:train_valid_index], X[train_valid_index:]
y_train_valid, y_test = y[:train_valid_index], y[train_valid_index:]

# Second split: separate out the validation set from the remaining data
train_index = int(len(X_train_valid) * train_ratio / (train_ratio + valid_ratio))
X_train, X_valid = X_train_valid[:train_index], X_train_valid[train_index:]
y_train, y_valid = y_train_valid[:train_index], y_train_valid[train_index:]

print(X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape)


(1000, 7, 5) (188, 7, 5) (63, 7, 5) (1000, 1) (188, 1) (63, 1)


Then we convert the matrices into tensor objects

In [11]:
X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train).float()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test).float()
X_valid, y_valid = torch.tensor(X_valid).float(), torch.tensor(y_valid).float()

Now we define a class that we will use to turn these individual matrices into train and test datasets

In [12]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]
    
train_dataset = TimeSeriesDataset(X_train, y_train)
valid_dataset = TimeSeriesDataset(X_valid, y_valid)
test_dataset = TimeSeriesDataset(X_test, y_test)

Now we wrap our datasets in data loaders

In [13]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Send the batches to our compute device (GPU:0 in this case as we defined in the 'setup' section)

In [14]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([16, 7, 5]) torch.Size([16, 1])


## Correlation Analysis

This is a simple yet effective method to determine if a feature should be included in the model. Correlation measures the degree of relationship between two variables. If a feature is highly correlated with the target variable, it's often beneficial to include it in the model.

In [16]:
# Select t-1 timestep for each feature
t_1_close = shifted_close_np_scaled[:, -1]
t_1_open = shifted_open_np_scaled[:, -1]
t_1_high = shifted_high_np_scaled[:, -1]
t_1_low = shifted_low_np_scaled[:, -1]
t_1_volume = shifted_volume_np_scaled[:, -1]

# Combine into a 2D array
data_t_1 = np.stack((t_1_close, t_1_open, t_1_high, t_1_low, t_1_volume), axis=-1)

# Create a dataframe
df_t_1 = pd.DataFrame(data_t_1, columns=['Close', 'Open', 'High', 'Low', 'Volume'])

# Calculate the correlation matrix
correlation_matrix_t_1 = df_t_1.corr()
print(correlation_matrix_t_1)


           Close      Open      High       Low    Volume
Close   1.000000  0.998624  0.999374  0.999384 -0.030139
Open    0.998624  1.000000  0.999446  0.999430 -0.032336
High    0.999374  0.999446  1.000000  0.999323 -0.021220
Low     0.999384  0.999430  0.999323  1.000000 -0.040430
Volume -0.030139 -0.032336 -0.021220 -0.040430  1.000000


## Permutation Importance

This method involves training the model with all the features first, then for each feature, the values are permuted in the validation data and the decrease in the model score is computed. The features which cause the largest decrease in score are considered the most important.

First, we want to understand the importance of multiple forms of time series data, i.e. open, close, high, low and volume on the performance of an LSTM model that we have already trained.

As a result, the first step is to load in the model.

In [17]:
# Optimal Parameters
hidden_size_optimal = 128
num_stacked_layers_optimal = 2
dropout_optimal = 0.0

# Fixed parameters
input_size = 5
num_epochs = 20
loss_function = nn.MSELoss()

# Define custom LSTM class (same as before)
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, 
                            batch_first=True, dropout=dropout)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model
model_optimal = LSTM(input_size, hidden_size_optimal, num_stacked_layers_optimal, dropout_optimal)
model_optimal.to(device)

# Load the saved model
model_optimal.load_state_dict(torch.load("../Models/MvLSTM/24-07-2023_14-42-13/MvLSTM.pth"))    # choose which saved model to load

# Ensure the model is in evaluation mode
model_optimal.eval()

LSTM(
  (lstm): LSTM(5, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

New we have loaded the LSTM model, we can implement our permutation importance procedure using the eli5 library

**Note:** Our features are 'Close', 'Open', 'High', 'Low', 'Volume'

In [18]:
def score(X, y):
    model_optimal.eval()
    X, y = X.to(device), y.to(device)
    with torch.no_grad():
        y_pred = model_optimal(X).detach().cpu().numpy()
    return np.sqrt(mean_squared_error(y.cpu().numpy(), y_pred))

baseline = score(X_valid, y_valid)
imp = []
for feature in range(X_valid.shape[2]):  # Iterate over features, not sequence steps
    save = X_valid[:, :, feature].clone()  # Use clone for PyTorch tensor
    X_valid[:, :, feature] = torch.tensor(np.random.permutation(X_valid[:, :, feature].cpu().numpy()), device=device)  # Convert back to tensor after permutation
    m = score(X_valid, y_valid)
    X_valid[:, :, feature] = save  # Restore original values
    imp.append(baseline - m)

# Print the feature importance
print('Feature importance:', imp)

Feature importance: [-0.06099544, -0.02909847, -0.042314332, -0.0385322, 3.360957e-05]


Permutation feature importance is a technique used to determine the most important features in our dataset. It works by shuffling the values of each feature and measuring how much the performance of the model decreases. The more the performance decreases, the more important the feature is.

Now to interpret these specific results:

* **Close:** 
* **Open:**
* **High:**
* **Low:**
* **Volume:**